In [ ]:
import keras
import tensorflow as tf

import os
tf.config.optimizer.set_jit(False)  # Desativa XLA

In [ ]:
from segmentandoDatasets import *

segmentando_datasets()

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from typing import List, Tuple
import os

def preprocessamento(caminho: str, proporcao_treino: float = 0.6, proporcao_teste: float = 0.2, proporcao_validacao: float = 0.2, autoencoder: bool = False):
    """
    Ao passar um dataFrame .csv, ele irá retornar geradores de dados para treino, teste e validação + os 3 .csv dividos igualmente os geradores.
    
    Parâmetros:
        caminho (str): Caminho para o arquivo CSV.
        proporcao_treino (float): Proporção de dados de treino.
        proporcao_teste (float): Proporção de dados de teste.
        proporcao_validacao (float): Proporção de dados de validação.
        autoencoder (bool): Se True, prepara os dados para um autoencoder (class_mode='input').
                            Se False, prepara os dados para classificação binária (class_mode='binary').
    
    Retorna:
        treino_gerador, validacao_gerador, teste_gerador, treino, teste, validacao
    """
    dataframe = pd.read_csv(caminho)

    treino, teste = train_test_split(dataframe, test_size=proporcao_teste, random_state=42)
    treino, validacao = train_test_split(treino, test_size=proporcao_validacao / (1 - proporcao_teste), random_state=42)

    img_width, img_height = 64, 64
    batch_size = 32

    def normalize_image(img):
        return img / 255.0

    treino_datagen = ImageDataGenerator(preprocessing_function=normalize_image)
    validacao_datagen = ImageDataGenerator(preprocessing_function=normalize_image)
    teste_datagen = ImageDataGenerator(preprocessing_function=normalize_image)

    class_mode = 'input' if autoencoder else 'binary'

    treino_gerador = treino_datagen.flow_from_dataframe(
        dataframe=treino,
        x_col='caminho_imagem',
        y_col='caminho_imagem' if autoencoder else 'classe', #Usar a imagem como saída se for autoencoder
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=class_mode,  #Class mode baseado no parâmetro autoencoder
        shuffle=False
    )

    validacao_gerador = validacao_datagen.flow_from_dataframe(
        dataframe=validacao,
        x_col='caminho_imagem',
        y_col='caminho_imagem' if autoencoder else 'classe',  
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=class_mode, 
        shuffle=False
    )

    teste_gerador = teste_datagen.flow_from_dataframe(
        dataframe=teste,
        x_col='caminho_imagem',
        y_col='caminho_imagem' if autoencoder else 'classe',  
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=class_mode, 
        shuffle=False
    )

    return treino_gerador, validacao_gerador, teste_gerador, treino, teste, validacao

In [ ]:
csv_file = 'Datasets_csv/df_PUC.csv'
treino_gerador, validacao_gerador, teste_gerador, treino, teste, validacao = preprocessamento(csv_file, autoencoder=True)

In [ ]:
autoencoder = tf.keras.models.load_model('/home/lucas/PIBIC (copy)/Modelos_keras/Autoencoder.keras')
autoencoder.load_weights('/home/lucas/PIBIC (copy)/weights_finais/Autoencoder.weights.h5')

In [ ]:
autoencoder.summary()

In [7]:
decoder = autoencoder.get_layer('sequential')
encoder = autoencoder.get_layer('sequential_1')

In [ ]:
# Obtém o primeiro batch
batch = next(iter(teste_gerador))

# Verifica a forma dos dados e das labels
print(f'Forma do batch de dados: {batch[0].shape}')
print(f'Forma das labels: {batch[1].shape}')


In [ ]:
imagens_codificadas = encoder.predict(teste_gerador, batch_size=8)

print(f'Forma de encoded_imgs: {imagens_codificadas.shape}')

imagens_decodificadas = decoder.predict(imagens_codificadas, batch_size=8)

In [28]:
import matplotlib.pyplot as plt
import numpy as np

def plot_autoencoder(x_test, Autoencoder):
    def normalize(image):
        image = np.clip(image, 0, 1)  # Garante que a imagem esteja no intervalo [0, 1]
        return (image - image.min()) / (image.max() - image.min()) if image.max() != image.min() else image

    plt.figure(figsize=(16, 8))

    for i in range(8):
        # Imagem original
        plt.subplot(2, 8, i + 1)
        plt.imshow(x_test[i])
        plt.title("Original")
        plt.axis("off")

        # Predição e normalização
        pred = Autoencoder.predict(x_test[i].reshape((1, 64, 64, 3)))
        pred_img = normalize(pred[0])

        plt.subplot(2, 8, i + 8 + 1)
        plt.imshow(pred_img)
        plt.title("Reconstruída")
        plt.axis("off")

    plt.show()


In [ ]:
"""_, _, _, x_train_UFPR04, y_train, x_test, y_test, x_val, y_val = preprocessamento("Datasets_csv/df_UFPR04.csv", 0.6, 0.2, 0.2)
_, _, _, x_train_UFPR05, y_train, x_test, y_test, x_val, y_val = preprocessamento("Datasets_csv/df_UFPR05.csv", 0.6, 0.2, 0.2)"""

In [30]:
x_teste, _ = next(teste_gerador)

In [ ]:
plot_autoencoder(x_teste, autoencoder)
"""plot_autoencoder(x_train_UFPR04, autoencoder)
plot_autoencoder(x_train_UFPR05, autoencoder)"""

In [ ]:
y_teste = teste['classe']
class_mapping = {'Empty': 0, 'Occupied': 1}

y_teste_numerico = y_teste.map(class_mapping)

print("print y:", y_teste_numerico)
print("shape de y_teste:", y_teste_numerico.shape)

In [33]:
shape_imagem_codificada = imagens_codificadas.reshape((imagens_codificadas.shape[0], -1))

In [ ]:
classificador = Sequential([
    Dense(128, activation='relu', input_shape=(2048,)),  # Ajuste a entrada conforme o tamanho do vetor
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída para classificação binária
])


In [35]:
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
classificador.fit(shape_imagem_codificada, y_teste_numerico, epochs=10, batch_size=16, validation_split=0.1)

In [ ]:
classificador.save("Modelos_keras/classificador.h5")
classificador.save_weights("weights_finais/classificador.weights.h5")

In [38]:
def preprocessamento_completo(caminho: str):
    """
    Prepara e retorna geradores de dados e dados em formato de arrays numpy para o conjunto inteiro.
    """

    nome_arquivo = os.path.basename(caminho)

    # Removendo a extensão e pegando a parte desejada
    nome_parte = os.path.splitext(nome_arquivo)[0].split('_')[1]

    dataframe = pd.read_csv(caminho)
    
    dataframe['classe'] = dataframe['classe'].astype(str)

    img_width, img_height = 256, 256
    batch_size = 32

    def normalize_image(img):
        return img / 255.0

    datagen = ImageDataGenerator(preprocessing_function=normalize_image)

    gerador = datagen.flow_from_dataframe(
        dataframe=dataframe,
        x_col='caminho_imagem',
        y_col='classe',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False  # Não embaralhar para garantir a ordem dos dados
    )

    def extrair_dados(gerador):
        x = []
        y = []
        for img_batch, label_batch in gerador:
            x.append(img_batch)
            y.append(label_batch)
            if len(x) * gerador.batch_size >= gerador.samples:
                break
        return np.concatenate(x), np.concatenate(y)

    x_dados, y_dados = extrair_dados(gerador)

    return nome_parte, x_dados, y_dados

def plot_predictions(images, predictions, true_labels, save_path:str, class_names, num=5):
        plt.figure(figsize=(10, 10))
        for i in range(num):
            plt.subplot(1, num, i + 1)
            plt.imshow(images[i])
            
            # Garantir que predictions e true_labels são inteiros
            pred_class = int(predictions[i])
            true_class = class_names.index(true_labels[i])
            
            plt.title(f'Pred: {class_names[pred_class]}\nTrue: {class_names[true_class]}')
            plt.axis('off')
        plt.savefig(save_path)
        plt.show()

In [39]:
def carregar_e_preprocessar_imagens(caminho_imagens, tamanho=(64, 64)):
    imagens = []
    for caminho in caminho_imagens:
        img = load_img(caminho, target_size=tamanho)
        img_array = img_to_array(img) / 255.0
        imagens.append(img_array)
    return np.array(imagens)

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from visualizacao import plot_confusion_matrix

def carregar_e_preprocessar_imagens(caminho_imagens, tamanho=(64, 64)):
    imagens = []
    for caminho in caminho_imagens:
        img = load_img(caminho, target_size=tamanho)
        img_array = img_to_array(img) / 255.0  # Normalizar para intervalo [0,1]
        imagens.append(img_array)
    return np.array(imagens)

def predizer_classes(classificador, encoder, caminho:str):
    nome_arquivo = os.path.basename(caminho)
    nome = os.path.splitext(nome_arquivo)[0].split('_')[1]

    dataframe = pd.read_csv(caminho)
    dataframe['classe'] = dataframe['classe'].astype(str)
    y_dados = dataframe['classe']
    class_mapping = {'Empty': 0, 'Occupied': 1}

    y_num = y_dados.map(class_mapping)

    x_dados = dataframe['caminho_imagem']

    imagens = carregar_e_preprocessar_imagens(x_dados)
    
    imagem_codificada = encoder.predict(imagens)
    flatten_imgs = imagem_codificada.reshape((imagem_codificada.shape[0], -1))

    predicao = classificador.predict(flatten_imgs)
    predicao_classes = predicao.argmax(axis=1)

    # Passar as imagens reais para plotar as previsões
    plot_predictions(imagens[:10], predicao_classes[:10], y_dados[:10], f"Resultados/Predicoes/Autoencoder_{nome}", class_names=['Empty', 'Occupied'])
    plot_confusion_matrix(y_num, predicao_classes, ['Empty', 'Occupied'], f"Resultados/Matriz_de_confusao/Autoencoder/Autoencoder_{nome}", f"Autoencoder {nome}")

predizer_classes(classificador, encoder, 'Datasets_csv/df_UFPR04.csv')
predizer_classes(classificador, encoder, 'Datasets_csv/df_UFPR05.csv')
predizer_classes(classificador, encoder, 'Datasets_csv/df_PUC.csv')
